In [2]:
import cv2
import os
from time import time
import glob
import h5py
import pandas as pd
from math import radians, degrees, atan2, sqrt

In [3]:
os.chdir("/home/donghan/DeepLabCut/data/")
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[300:] 

In [ ]:
def videoWriter(task, filename, outputName, corrX, corrY, distX, distY, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (distY, distX)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out
    
def videoCropper(filename, outputName, corrX, corrY, distX, distY, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + distX
    corrY1 = corrY + distY

    video = videoWriter("edited", filename, outputName, corrX, corrY, distX, distY, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

    
def getCoord(data):
    '''
    Task: Obtaining the coordinates of cropped points
    
    data: DataFrame
    Mouse moving location, by pixel
    
    !!! We are assuming that the mouse at least reaches the boundary once
    '''
    x = sorted(data['x'], reverse = True)
    y = sorted(data['y'], reverse = True)
    
    # Try to exclude outliers
    for i in range(10):
        xMax = round(x[i + 1]) if x[i] - x[i + 1] >= 1 else round(x[i])
        
        xMin = round(x[-i-1]) if x[-i] - x[-i-1] >= 1 else round(x[-i])
            
        yMax = round(y[i + 1]) if y[i] - y[i + 1] >= 1 else round(y[i])

        yMin = round(y[-i-1]) if y[-i] - y[-i-1] >= 1 else round(y[-i])
    
    lengthX = xMax - xMin
    lengthY = yMax - yMin
    
    
    return xMin, yMin, lengthX, lengthY



def dataModifier(data):
    '''
    Task: Modify the location data based on the cropped video
    
    data: DataFrame
    '''
    
    x, y, lengthX, lengthY = getCoord(data)
    filtered = data.copy(deep = True)
    filtered = filtered[(filtered['y']>x) & (filtered['x']>y)]
    filtered['x'] = filtered['x'] - y
    filtered['y'] = filtered['y'] - x
    filtered['coords'] = list(range(1,len(filtered)+1))
    filtered.index = list(range(1,len(filtered)+1))
    
    return filtered

In [ ]:
def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    return (direction, degreeL)
# Whether the mouse's direction towards bullying mouse or not, 1: yes; 0: no
# The degree of head direction, clockwise.


## Calculate Angle and Distance between enclosure and defeated mouse

In [94]:
def quadrant(a, b): 
    '''
    Task: Check if two points are in the same quadrant
    '''
    if ((a[0] > 0 and b[0] > 0) or (a[0] < 0 and b[0] < 0)) and ((a[1] > 0 and b[1] > 0) or (a[1] < 0 and b[1] < 0)):
            return True
    else:
        return False
    
def getAngle(a, b, c):
    '''
    Task: Calculate angle between three points. 
    
    PARAMETERS
    -----------
    a, b, c: tuple
        Three points in the coordinate system
    '''
    cb = atan2(c[1]-b[1], c[0]-b[0])
    ab = atan2(a[1]-b[1], a[0]-b[0])

    # If they are in the same quadrant: acute angle
    if quadrant(c, b) and quadrant(a, b):
        angle = degrees(cb - ab)
    else:
        # Obtuse angle or in different quadrant
        rad360 = radians(360)
        # cd or ab < 0 means they over 180 degree
        # +ran360 produce a positive radian representing degrees over 180
        cb = rad360 + cb if cb < 0 else cb
        ab = rad360 + ab if ab < 0 else ab
        # Order of points may changed in a motional situation, if exceed 180, adjust it by 360-
        angle = 360 - abs(degrees(cb - ab)) if abs(degrees(cb - ab)) > 180 else abs(degrees(cb - ab))
            
    return angle + 360 if angle < 0 else angle

def getDist(a, b):
    '''
    Task: Calculate distance
    '''
    dist = sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2)  
    return dist  

In [73]:
def calDist_Angle(data, enclosure):
    '''
    Task: Calculate distance and angle between enclosure and defeated mouse
    reference point is the center point of head of tail regarding Angle
    head point for Distance
    
    Parameters
    ----------
    data: DataFrame
        Read location data
    
    enclosure: tuple
        enclosure's coordinate, ex. (68,204)
    '''
    head = pd.concat([data["x"], data["y"]],axis=1)
    tail = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    # Head--------------------Tail-----------
    mid = (head.x + tail.x) / 2, (head.y + tail.y) / 2
    # Avoid Pandas KeyError
    mid = np.array(mid)
    x = np.array(head.x)
    y = np.array(head.y)
    
    angle = []
    dist = []
    for i in range(len(head)):
        # Set middle point of the mouse to be (0,0), then calculate the corresponding relative converted coordinates
        # Order: head relative location, central point, enclosure relative location
        angle.append(getAngle((x[i] - mid[0][i], y[i] - mid[1][i]),(0, 0), (enclosure[0] - mid[0][i], enclosure[1] - mid[1][i])))
        dist.append(getDist((mid[0][i], mid[1][i]), enclosure))
    return angle, dist

## Coordinate System Transformation

In [8]:
import numpy as np
import cv2
 
def orderPoints(pts):
    '''
    Task: sort points by the following order: top-left, top-right, bottom-left, bottom-left
    
    PARAMETERS:
    -----------
    pts: list 
        list of tuple of points coordinate
    '''
    
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

def fourPointTransform(pts, image = None):
    '''
    Task: Transform image by its corner four points. 
    
    PARAMETERS:
    -----------
    image: array, Optional if intend to transform an image
        image matrix
    
    pts: list
        list of tuple of four corner points
    '''
    rect = orderPoints(pts)
    (tl, tr, br, bl) = rect

    # Transformed width
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # Transformed height
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    # Specify the destination points based on the dimensions: top-left, top-right, bottom-right, and bottom-left
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    if image is not None:
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        return warped, M
    return M



In [130]:
pts = np.array([(28,39),(74,387),(364,23),(377,336)])
img = cv2.imread("test2.jpg")
cv2.imwrite("test10.jpg",fourPointTransform(img, pts)[0])

True

In [9]:
def locDataConvert(data, pts):
    '''
    Task: Convert mouse location data to prospective coordicates after correcting coord system.
    
    PARAMETERS:
    -----------
    pts: list
        list of four corner points of the video
    '''
    # Transformation matrix
    matrix = fourPointTransform(pts)
    # x, y, 1
    head = pd.concat([
        data["x"], 
        data["y"], 
        pd.DataFrame([1]*len(data))],axis = 1).values.transpose()
    tail = pd.concat([
        data["x.1"], 
        data["y.1"], 
        pd.DataFrame([1]*len(data))], axis = 1, keys=['x', 'y', 'constant']).values.transpose()
    
    # Dot product of transformed matrix and head, tail data
    transformedHead = np.dot(matrix, head)
    transformedTail = np.dot(matrix, tail)
    
    tHeads = []
    tTails = []
    for i in range(transformedHead.shape[1]):
        tempx = []
        tempy = []
        for x,y in zip(transformedHead, transformedTail):
            tempx.append(int(round(x[i])))
            tempy.append(int(round(y[i])))
        tHeads.append(tempx[:2])
        tTails.append(tempy[:2])
    
    return tHeads, tTails

In [131]:
for p in i:
    print(p)

[239.5114147540263, 410.0215598784492, 0.7687282287273324]
[253.58444120196714, 411.5592877616764, 0.7628687858648031]
[-79.24305769822806, 385.8102816363677, 0.8983081233627928]
[-79.45472755845108, 385.60873443300443, 0.8984489592759097]
[280.20231389672153, 410.91553899509404, 0.7528355049977586]
[253.52666357389376, 411.02083127926903, 0.7630500380321306]
[-78.94368057624192, 385.49642611474206, 0.8982858513796199]
[-79.60916674821604, 385.5528947353368, 0.8985247717125633]
[-50.38616527143955, 392.5454525402606, 0.8852351318539069]
[-79.49800571994041, 385.3546075867187, 0.8985406510747538]
[253.71241397361098, 411.1625086397798, 0.7629368429546073]
[-79.88241329340676, 385.19229524856144, 0.8987362423945795]
[-79.4091953412561, 385.5391710287169, 0.8984520203018685]
[-79.38751755471671, 385.38151346235696, 0.8984902665272867]
[-78.83370076381362, 385.4917810462549, 0.8982449822757098]
[-79.34925498468182, 385.3982129803453, 0.898470637502019]
[-78.80628998642119, 385.100442290687

[48.38440093718059, 74.47775984654601, 0.9412567927329818]
[52.918552779643335, 69.40410368867975, 0.9410140720753597]
[59.835833322298356, 68.00912283222135, 0.9387693486430355]
[63.9972292714145, 68.10086418688226, 0.9371439309292375]
[72.47914395838967, 68.11436467193101, 0.9338821923005435]
[81.52521744010393, 67.18578514178537, 0.930682062241879]
[92.70141394866916, 67.95558882179105, 0.926162085468379]
[103.92445142850693, 70.66318759089134, 0.921051690457218]
[113.09575474782947, 69.92624692006726, 0.9177468514793811]
[122.72037111226633, 70.54800235128147, 0.9138665423421508]
[133.8958332693424, 71.4452398291991, 0.9093092034920834]
[146.37595537124355, 76.16971597996519, 0.9031201964453027]
[159.16600206325563, 79.3931132613233, 0.8972555738125605]
[166.65752759285846, 78.01091834108001, 0.8947865162460786]
[176.82581312084466, 78.77758827187301, 0.8906545856291785]
[189.64611706359926, 81.21742609601691, 0.8850098063257635]
[202.8171866835031, 85.2172076393314, 0.878769494622

[14.486327800919987, 245.0482424629184, 0.9038896650662079]
[14.172835836865254, 243.44625024548256, 0.9044833024917601]
[13.996523508727854, 244.062084851052, 0.9043691024485184]
[13.181970222308387, 243.2402647217895, 0.9049247245372871]
[12.491768056134248, 243.87272294546958, 0.9050029896784831]
[13.0721995279332, 244.35527375072908, 0.9046375102069903]
[13.575297922551322, 243.76099648756545, 0.9046198295415543]
[13.554816917687972, 241.57486118614898, 0.905273483448166]
[14.386727534182057, 242.28450174460144, 0.9047443327287861]
[14.44781038014159, 238.5763419759813, 0.9058162676535699]
[18.20246498905503, 234.74760851659246, 0.905505185873804]
[20.598482714167375, 230.51650291601706, 0.9058347933460386]
[22.586652512552263, 227.78586591504708, 0.9058778072675854]
[25.206732252048383, 220.27402340664347, 0.9070904909736253]
[26.5340668890274, 217.70728301747056, 0.9073389044752018]
[26.490686974545824, 214.4263642268483, 0.9083247540623512]
[27.741609068305493, 204.0571904946822

[89.59783607798458, 261.97513510989484, 0.8700404599276699]
[89.6505554501275, 264.17212413991535, 0.8693712176381148]
[90.76829275484467, 263.07798358903824, 0.8692651255809358]
[90.67845015943546, 263.80923041601073, 0.8690836211350579]
[88.2486668814713, 263.1706133266784, 0.8702055055108071]
[88.7654918526476, 263.20355256105375, 0.8699972720565051]
[89.99549612867278, 262.2012124328548, 0.8698209423016879]
[88.6588287515085, 262.66190839484375, 0.8701982418993923]
[87.92692345467893, 264.64898167685783, 0.8698923692190752]
[88.21628607466167, 264.6003702334305, 0.869795590142294]
[89.70271395199205, 265.07086554918783, 0.8690856949887497]
[89.8763251201768, 263.0872593679407, 0.8696049741954535]
[90.5976177023839, 264.5926579697988, 0.8688832417954446]
[88.71132000463624, 264.02400289794787, 0.8697757162739117]
[80.86758971829043, 263.9752949666619, 0.8728027401064131]
[76.5482701804077, 259.32169443764263, 0.8758363932055332]
[76.06170796265336, 256.9963449770343, 0.8767101845695

In [180]:
start = time()
pts = np.array([(28,39),(74,387),(364,23),(377,336)])
i = locDataConvert(data,pts)[0]
[p[0] for p in i]

[240,
 254,
 -79,
 -79,
 280,
 254,
 -79,
 -80,
 -50,
 -79,
 254,
 -80,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 253,
 -79,
 -79,
 254,
 254,
 -79,
 -79,
 -79,
 -79,
 -79,
 254,
 -79,
 -79,
 -80,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 185,
 -79,
 225,
 -79,
 281,
 -79,
 225,
 -79,
 254,
 -80,
 -79,
 -80,
 254,
 -80,
 -50,
 -78,
 -79,
 -79,
 -79,
 -51,
 -79,
 225,
 225,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -78,
 -79,
 -50,
 -79,
 -50,
 -79,
 185,
 184,
 -79,
 -79,
 -79,
 -80,
 254,
 -80,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -50,
 -79,
 -78,
 -80,
 -50,
 -79,
 -79,
 -80,
 -78,
 -79,
 185,
 -79,
 -78,
 -79,
 -78,
 199,
 199,
 -79,
 185,
 -80,
 198,
 -79,
 -79,
 -79,
 -79,
 -79,
 199,
 -79,
 -79,
 -80,
 192,
 199,
 -79,
 -79,
 -79,
 -79,
 185,
 -79,
 185,
 -79,
 -79,
 -79,
 -78,
 -79,
 -79,
 -78,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79,
 -79

In [ ]:
csvfile

In [ ]:
test = move_data
test

In [ ]:
import numpy as np
pd.options.mode.chained_assignment = None
p['coords'] = list(range(1,len(p)+1))
p.index = list(range(1,len(p)+1))

In [ ]:
p

In [ ]:
p = test[(test['y']>54) & (test['x']>38)]

In [ ]:
test = move_data['x'] - 38
test[test>0]

In [29]:
def videoWriter(task, filename, outputName, corrX = None, corrY = None, dist = 320, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (dist, dist)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out

def fieldOfView(data, filename, outputName, radius, rotationAngle, color, viewRange, thickness, displayVideo = False):
    '''    
    Task: draw sector for field of view

    All input parameters must be Int

    PARAMETERS:

    data: mouse movement that read from .h5 file, including head and tail. 

    angle: 0, rotation angle

    radius: radius of the sector, millimeter

    color: sector line color

    viewRange: range that the mouse has vision on

    thickness: the thickness of the sector line

    '''

    #Obtain coordinates data for head and tail, as well as the head direction by degrees
    

#     head = zip(round(data['x']).astype(int), round(data['y']).astype(int))
#     tail = zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int))
    
    
    
#     degrees = head_dir(data)[1]
    
    #Get pixel to mm conversion
#     x, y, lengthX, lengthY = getCoord(data)
#     length = lengthX if lengthX < lengthY else lengthY
#     convert = 440/length # video side length 440mm, square
#     radius = round(radius / convert)
    
    
    #cap: Read video
    #out: Write video
    video = videoWriter("field of view", filename, outputName, displayVideo)
    cap = video[0]
    out = video[1]

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    while(cap.isOpened()):
        for center, angle in zip(data, degrees):
            ret,img = cap.read()
        #draw the sector representing the field of view of mouse
            angle = int(round(angle))
            startAngle = angle - viewRange
            endAngle = angle + viewRange
            img2 = drawSector(img, radius, tuple(center), rotationAngle, startAngle, endAngle, color, thickness)
            if count == 300:
                print(img2, type(img2))
            out.write(img2)

            if displayVideo == True:
                cv2.imshow('Draw field of view video',img2) 

            # Break the loop when the last frame has been rotated
            if count == frameCnt:
                print (filename, 'successfully ', "added field of view!!!")
                return
            else:
                count += 1
        if k==27:
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [43]:

def transform(filename, outputName, displayVideo = False):
    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + "transformed"
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    file = "./" + "transformed" + "/" + outputName

        
        
#     video = videoWriter("transformed", filename, outputName, displayVideo)
#     cap = video[0]
#     out = video[1]

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    while(cap.isOpened()):
        ret,img = cap.read()
        img2 = fourPointTransform(pts, img)[0]
        if count == 1:
            out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (fourPointTransform(pts, img)[0].shape[1], fourPointTransform(pts, img)[0].shape[0]))

        out.write(img2)

        if displayVideo == True:
            cv2.imshow('Draw field of view video',img2) 

        # Break the loop when the last frame has been rotated
        if count == frameCnt:
            print (filename, 'successfully ', "added field of view!!!")
            return
        else:
            count += 1
#         if k==27:
#             k=cv2.waitKey(30) & 0xff
#             #once you inter Esc capturing will stop
            
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [ ]:
PROBLEM: LOCATION DATA TRANSFORMATION IS NOT ACCURATE, NEED FIX

In [36]:
fourPointTransform(pts, img)[0].shape

(351, 336, 3)

In [34]:
img = cv2.imread("temp.jpg")
fourPointTransform(pts, img)[0]

array([[[179, 191, 203],
        [179, 191, 203],
        [179, 191, 203],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[180, 191, 205],
        [179, 190, 204],
        [178, 189, 203],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[179, 190, 204],
        [178, 189, 203],
        [178, 189, 203],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [7]:
cv2.imwrite("temp.jpg",fourPointTransform(pts, img)[0] )

NameError: name 'fourPointTransform' is not defined

In [42]:
pts = np.array([(28,39),(74,387),(364,23),(377,336)])

transform("1034 SI_B, Aug 15, 13 7 49.mp4", "1034 SI_B, Aug 15, 13 7 49 transformed.mp4")

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068


In [26]:
degrees = head_dir(data)[1]


In [176]:
[round(i) for i in locDataConvert(data,pts)[0]]

TypeError: type list doesn't define __round__ method

In [175]:
for i, j in zip(int(locDataConvert(data,pts)[0]), degrees):
    print(i, j)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [44]:
fieldOfView(locDataConvert(data,pts)[0],"1034 SI_B, Aug 15, 13 7 49 transformed.mp4", "transformTest.mp4", 50, 0, BLACK, 60, 1)

[[[174 184 190]
  [170 180 186]
  [168 178 184]
  ...
  [165 169 175]
  [160 164 170]
  [156 160 166]]

 [[177 187 193]
  [172 182 188]
  [170 180 186]
  ...
  [163 167 173]
  [159 163 169]
  [155 159 165]]

 [[174 184 190]
  [172 182 188]
  [171 181 187]
  ...
  [163 167 173]
  [158 162 168]
  [155 159 165]]

 ...

 [[183 197 213]
  [184 198 214]
  [184 198 214]
  ...
  [184 194 206]
  [182 192 204]
  [181 191 203]]

 [[183 197 213]
  [184 198 214]
  [185 199 215]
  ...
  [184 194 206]
  [182 192 204]
  [180 190 202]]

 [[183 197 213]
  [184 198 214]
  [185 199 215]
  ...
  [183 193 205]
  [181 191 203]
  [180 190 202]]] <class 'numpy.ndarray'>
1034 SI_B, Aug 15, 13 7 49 transformed.mp4 successfully  added field of view!!!


In [24]:
BLACK = (0,0,0)
from math import degrees, atan2
def drawSector(image, radius, center, angle, startAngle, endAngle, color, thickness):
    # Ellipse parameters
    axes = (radius, radius)
    cv2.ellipse(image, center, axes, angle, startAngle, endAngle, color, thickness)
    points = cv2.ellipse2Poly(center, axes, angle, startAngle, endAngle, thickness)
    point1 = tuple(points[0]) # End's coordinates
    point2 = tuple(points[-1]) # Another end's coordinates
    cv2.line(image, center, point1, color)
    cv2.line(image, center, point2, color)
    return image

def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    if __name__ == "__main__": 
        return (direction, degreeL)

In [ ]:
x = dataModifier(move_data)

In [ ]:
fieldOfView(x, '1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest1.mp4", 50, 0, BLACK, 60, 1)

In [ ]:
x, y, lengthX, lengthY = getCoord(move_data)

In [ ]:
x, y, lengthX, lengthY

In [ ]:
convertX = 440/lengthX
convertY = 440/lengthY
print(convertX, convertY)

In [ ]:
440/331

In [ ]:
videoCropper('1034 SI_B, Aug 15, 13 7 49 rotated.mp4', "cropTest rotated.mp4", y, x, lengthY, lengthX)

In [ ]:
##Question: choose length of the edge in order to be converted in video processing